# BFS

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from othello.OthelloGame import OthelloGame as Game
from othello.OthelloGame import display as displayGame
from matplotlib import pyplot as plt
from matplotlib import patches
import numpy as np
from tree_search_algs import bfs_depth, bfs_cannonical, bfs_no_depth

In [3]:
n = 4
game = Game(n)
board = game.getInitBoard()

In [5]:
%time states_depth = bfs_depth(game, board, verbose=1)

Profundidad: 1
Estados hasta el momento: 1
Time: 0.0
Profundidad: 2
Estados hasta el momento: 5
Time: 0.0
Profundidad: 3
Estados hasta el momento: 17
Time: 0.01
Profundidad: 4
Estados hasta el momento: 59
Time: 0.06
Profundidad: 5
Estados hasta el momento: 179
Time: 0.06
Profundidad: 6
Estados hasta el momento: 555
Time: 0.19
Profundidad: 7
Estados hasta el momento: 1603
Time: 0.64
Profundidad: 8
Estados hasta el momento: 4121
Time: 1.71
Profundidad: 9
Estados hasta el momento: 9769
Time: 3.67
Profundidad: 10
Estados hasta el momento: 19487
Time: 6.1
Profundidad: 11
Estados hasta el momento: 32295
Time: 13.1
Profundidad: 12
Estados hasta el momento: 43195
Time: 9.95
Profundidad: 13
Estados hasta el momento: 46317
Time: 7.57
Profundidad: 14
Estados hasta el momento: 46673
Time: 3.04
Profundidad: 15
Estados hasta el momento: 46689
Time: 0.32
Profundidad: 16
Estados hasta el momento: 46689
Time: 0.01
Profundidad: 17
Estados hasta el momento: 46689
Time: 0.0
CPU times: user 41.3 s, sys: 68

In [6]:
%time states_no_depth = bfs_no_depth(game, board)

CPU times: user 37.6 s, sys: 359 ms, total: 37.9 s
Wall time: 38.5 s


In [9]:
%time states_cannonical = bfs_cannonical(game, board)

CPU times: user 58.3 s, sys: 535 ms, total: 58.9 s
Wall time: 1min


In [10]:
len(states_depth), len(set(states_no_depth)), len(states_cannonical)

(46689, 58533, 53651)

# 6x6 (Analyse if it is possible because of number of states)
The keys have to be string because of shelve

In [4]:
n = 6
game = Game(n)
board = game.getInitBoard()

In [5]:
board_str = str(board.reshape(-1) + 1)[1:-1]

In [6]:
(np.array(board_str.split(' '), dtype=int) - 1).reshape(game.n, game.n)

array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0, -1,  1,  0,  0],
       [ 0,  0,  1, -1,  0,  0],
       [ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0]])

In [22]:
import os
import shelve
import collections
def bfs_cannonical(game, root, folder, first_player=1):
    if not os.path.exists(folder):
        os.makedirs(folder)
    state_str = str(root.reshape(-1) + 1)[1:-1]
    seen = set([])
    cannonical_states = shelve.open(folder+'/states', flag='n', writeback=True) 
    # cannonical_states = {}
    # deque es como una pila pero con doble entrada (rear-front)
    queue = collections.deque([state_str])
    while queue:
        vertex = queue.popleft()
        state = (np.array(vertex.split(' '), dtype=int) - 1).reshape(game.n, game.n)
        valid_moves = np.array(game.getValidMoves(state, 1))
        valid_moves = np.where(valid_moves == 1)[0]
        if game.getGameEnded(state, 1) == 0:
            cannonical_states[vertex] = {}
            for action in valid_moves:
                cannonical_states[vertex][action] = {}
                next_state, next_player = game.getNextState(state, 1, action)
                next_state = game.getCanonicalForm(next_state, -1)
                node = str(next_state.reshape(-1) + 1)[1:-1]
                cannonical_states[vertex][action]['winner'] = game.getGameEnded(next_state, 1) # * np.abs(next_state.sum())
                cannonical_states[vertex][action]['next_node'] = node
                if node not in seen:
                    seen.add(node)
                    queue.append(node)
        N = len(cannonical_states)
        if N%10000 == 0:
            print('\rstates: {}'.format(N), end='')
            cannonical_states.sync()
    print()
    cannonical_states.close()
    return states

In [ ]:
%time _ = bfs_cannonical(game, board, './data/6by6', first_player=1)

states: 1330000

In [1]:
1

1